# It's Not Convoluted, It's Convolution!

Phew, it's really been a whirlwind of activity up until this point. Before we go on, you should really take a moment to pat yourself on the back. So far, you've learned

* Lua
* Torch and Tensors
* the `nn` library
* how to classify MNIST digits with reasonable accuracy

that's quite the accomplishment! Well, you're 97% of the way there; just 2.7% to go, right? Let us forge onward!

Up until now, we've been treating an MNIST digit as a really, really big vector. This is odd because, as far as the network knows, each component/pixel is independent of the rest. Since the input is actually a 2D image, we know that this isn't the case! How, then, can we take advantage of this additional structure?

If we wanted to stick with linear layers, we could try connecting each $3\times 3$ neighborhood of pixels to a single output. In that case, if we had a $5\times 5$ image, we'd have four separate `nn.Linear` modules producing a $3\times 3$ output. That's not too bad except it assumes that the information learned about the top left neighborhood is independent from that learned about the bottom right. For instance, if we wanted to detect a 1, we'd need a vertical line detector; a repeated-linear network would have to learn this same detector for every position in the image!

Much better would be to, instead, learn a bunch of different $n\times n$ *filters* to slide (or "*convolve*") over each $n\times n$ neighborhood in the image. Now, you can have one filter that detects horizontal edges, another one that detects vertical edges, and possibly even one that detects the holes inside of 0s and 9s! The best part is that now you only have to learn $n^2 \cdot numFilters$ parameters instead of $n^2 \cdot (N-2)^2$, which speeds up computation and helps stave off overfitting.

If this doesn't make sense yet, don't fret. First, check out this diagram of the computation performed during an application of a convolutional filter.

<img src="https://i.stack.imgur.com/GvsBA.jpg">
<div class="figcaption">Figure 1: Computation performed during an application of a single conv filter.</div>

Still considering the image in Figure 1, to produce the entire *output feature map*, one must simply apply the conv filter at every valid location in the image. Note that, due to edge effects, the output feature map has size $o = N - n + 1$ (where $N$ is the input size and $n$ is the kernel/filter size).

If you grok the previous bit, this next part should be a simple extension: if you have $f$ filters, applying all of them to an input will give you $f$ output feature maps. You can then stack those up into a *volume* of size $f\times o \times o$ and feed them into the next conv layer. This is to say that, in general, the input to a single conv filter is an $f \times n \times n$ volume. In fact, you can think of an RGB image as a special case of a conv volume where each channel is one feature map (albeit one that is highly correlated with the others). 

Again, for you diagram people:

<img src="https://upload.wikimedia.org/wikipedia/commons/6/68/Conv_layer.png" style="width: 33%">
<div class="figcaption">Figure 2: Input and output conv volumes.</div>

As each feature map goes through more and more convolutional layers, the features become increasingly abstract. For example, your first layer might contain a horizontal and vertical edge detector. In the second layer, a filer could then combine the feature maps of the - and | detectors to yield a + detector! 

If diagrams aren't your style and you prefer the cold, hard code the module of the hour is [`nn.SpatialConvolution`](https://github.com/torch/nn/blob/master/doc/convolution.md#nn.SpatialConvolution):

In [1]:
require 'nn'
conv = nn.SpatialConvolution(3, 5, 3, 3) -- 1 feature map of size 28x28 -> 5 feature maps of size 26x26
print(tostring(conv))

nn.SpatialConvolution(3 -> 5, 3x3)	


In [2]:
img = torch.rand(2, 3, 28, 28) -- perhaps 2 MNIST images
conv:forward(img)
print(conv.output:size())

  2
  5
 26
 26
[torch.LongStorage of size 4]



And, of course, for you math/signals people: yes I know that the operation is actually cross-correlation; it's just *called* convolution...

### Taking a Page from Term Papers

One last thing before we slide on to the next part: if you want to capture details present at the edges of the image, the general consensus is that one should add a border of zeros known as *padding*. So, if we wanted to produce a $28 \times 28$ output in the previous example, we would do:

In [3]:
padW, padH = 1, 1
conv = nn.SpatialConvolution(3, 5, 3, 3, 1, 1, padW, padH) -- add a frame of width 1 before convolving
print(conv:forward(img):size())

  2
  5
 28
 28
[torch.LongStorage of size 4]



## Pool(ing) Party!!

I bet that all you want to do now is call it a day and go relax by a pool. Well, you're not done yet! We haven't even covered pooling!

Pooling, in a nutshell, throws out several pixels in a neighborhood in exchange for *translation invariance*, which is just a fancy way of saying that shifting your input won't significantly change your output. 

The two most common types of pooling are max and average pooling, both of which are available in Torch as [`nn.SpatalMaxPooling`](https://github.com/torch/nn/blob/master/doc/convolution.md#nn.SpatialMaxPooling) and [`nn.SpatialAveragePooling`](https://github.com/torch/nn/blob/master/doc/convolution.md#nn.SpatialAveragePooling).

Is it that time already for more diagrams?

<img src="https://upload.wikimedia.org/wikipedia/commons/e/e9/Max_pooling.png" style="width: 33%">
<div class="figcaption">Figure 3: Max pooling of a single feature map using a $2\times 2$ kernel and stride.</div>

Diagram! Code! Diagram! Code!

In [4]:
pool = nn.SpatialMaxPooling(2, 2) -- takes the max in a 2x2 neighborhood
print(tostring(pool))             -- slides by 2 px after each application (there's no overlap)

nn.SpatialMaxPooling(2x2, 2,2)	


In [5]:
img = torch.Tensor(1, 6, 6):random(99)
print(img)
print(pool:forward(img))
print(pool:backward(img, torch.ones(1, 3, 3))) -- mask of maximal pixels in each neighborhood

(1,.,.) = 
  52  64  61  37  60  51
  56  45  59  25  72  44
  24  14  31  34  33   1
  59  91  73  49   5   6
  27  17  77  54  51   6
  39  67  65  84  82  49
[torch.DoubleTensor of size 1x6x6]

(1,.,.) = 
  64  61  72
  91  73  33
  67  84  82
[torch.DoubleTensor of size 1x3x3]

(1,.,.) = 
  0  1  1  0  0  0
  0  0  0  0  1  0
  0  0  0  0  1  0
  0  1  1  0  0  0
  0  0  0  0  0  0
  0  1  0  1  1  0
[torch.DoubleTensor of size 1x6x6]



Let's see how translation invariance works. In this next example, the image will shift by one pixel to the right. Observe how the output feature map remains similar to the original!

In [6]:
shiftImg = torch.Tensor(1, 6, 6):random(99)
shiftImg:narrow(2, 2, 5):copy(img:narrow(2, 1, 5))
shiftPool = pool:clone()
print('Shifted: ')
print(shiftPool:forward(shiftImg))
print('vs Original:')
print(pool.output)

Shifted: 	
(1,.,.) = 
  64  97  60
  56  59  72
  91  77  51
[torch.DoubleTensor of size 1x3x3]

vs Original:	
(1,.,.) = 
  64  61  72
  91  73  33
  67  84  82
[torch.DoubleTensor of size 1x3x3]



Pretty close, right? Voila! Translation invariance.

#### Back to the Future

As a final note on pooling, most of the recent models like the [AllConvNet](https://arxiv.org/abs/1412.6806) and [ResNet](https://arxiv.org/abs/1603.05027) eschew pooling in favor of *strided* convolutions, which are like regular convolutions except they skip several pixels. The rationale is that fixed pooling can lead to artifiacts like aliasing and that a learned convolution lets the network decide how best to downsample.

Torch lets you do strided convolutions using the `dW` and `dH` parameters like so:

In [7]:
img = torch.rand(2, 3, 28, 28)
dW, dH = 2, 2
conv = nn.SpatialConvolution(3, 5, 3, 3, dW, dH) -- stride of 2 in each direction
print(tostring(conv))
print(conv:forward(img):size())

nn.SpatialConvolution(3 -> 5, 3x3, 2,2)	
  2
  5
 13
 13
[torch.LongStorage of size 4]



## It's All in Your Head, Redux

Probably the coolest part of the convolutional architecture is that it's quite strongly motivated by the structure and function of your [visual cortex](https://en.wikipedia.org/wiki/Visual_cortex#Primary_visual_cortex_.28V1.29). In particular, convolutions correspond to the operations performed by [simple cells](https://en.wikipedia.org/wiki/Simple_cell) and pooling is done by [complex cells](https://en.wikipedia.org/wiki/Complex_cell).

If you skim the linked articles a bit, you'll notice that the brain also takes into account temporal information. Since we're working with images right now, that's not really relevant, but if we were working with video, however, we could use a similar, convolutional architecture except you'd just bump up the dimensions by one! (@see [`nn.VolumetricConvolution`](https://github.com/torch/nn/blob/master/doc/convolution.md#nn.VolumetricConvolution))

## To 99.7% and Beyond!

With convolutions and pooling in hand, we can now go about improving our paltry 98% on MNIST to something that's actually acceptable.

We'll use the same training framework as before; the only thing that will change is the model, which you'll place in [`mnist/models/conv.lua`](../edit/mnist/models/conv.lua).

Whereas before we stacked some combination of `nn.Linear` and `nn.ReLU`, this time around, you'll want to do something more akin to [`nn.SpatialConvolution`](https://github.com/torch/nn/blob/master/SpatialConvolution.lua), [`nn.ReLU`](https://github.com/torch/nn/blob/master/doc/transfer.md#relu), and then throw in a [`nn.SpatialMaxPooling`](https://github.com/torch/nn/blob/master/doc/convolution.md#nn.SpatialMaxPooling) every so often.

Does this all seem very arbitrary yet? I certainly hope so! Despite convnets being very well established as the Right Thing™ to use for images and videos, there's still no principled way of designing one for a particular task. As it stands, one generally takes an existing architecture (or pre-trained network)and then *fine-tunes* (read: twiddles) it until it performs well on a specific task. If this state of affairs is unsatisfying to you, it's an open area of research that could use your efforts!

In any case, you know the drill: assemble a network, train, evaluate, rinse and repeat!

Hint 1: training will take longer than before, so make judicious use of filters per layer and liberally apply pooling/stride. If you need inspiration, try doing an image search for *LeNet-5*, the original network used on MNIST.

Hint 2: keep track of your feature map sizes after each convolution/pooling layer since you will need to ensure that the output has size $N \times 10$. Below is a quick function that computes the output size of a conv/pool layer. Also remember that, if you want to use a `nn.Linear` layer, you'll have to use an `nn.View`!

In [8]:
function outSize(inSize, kernelSize, stride, padSize)
    padSize = padSize or 0 -- default arguments
    stride = stride or 0
    
    o = math.floor((inSize + 2*padSize - kernelSize) / stride + 1)
    print(inSize..'x'..inSize..' -> '..o..'x'..o)
end

In [9]:
outSize(28, 3, 2)

28x28 -> 13x13	



As before, verify that your network has the correct input/output sizes:

In [ ]:
dofile('mnist/test/conv_io.lua') -- check the Tensor sizes!

Train to your heart's content! (Don't forget to twiddle hyperparameters!)

In [ ]:
trainMNIST = dofile('mnist/main.lua')
trainMNIST({modelType='conv', nEpochs=5})

# 🎉 Congratulations, you made it! 🎉

You've successfully reached the end of the prepared exercises! With any luck, you should now be able to convince yourself that you know how to do deep learning in Torch7.

Thanks for coming! I hope that you had fun :)